In [1]:
#Mount drive files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import io
import os
import pandas as pd
import numpy as np
##sentence tokeniser
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#get biopaper data 
abstract_path = 'drive/My Drive/Year5/csc4006 Final Project/data/abstract_id_label.csv'
a_df = pd.read_csv(abstract_path,sep='\t')
a_df = a_df.dropna()
a_df = a_df[a_df.label != 'unknown']
a_df.shape

(29312, 3)

In [3]:
#get biopaper data 
mtext_path = 'drive/My Drive/Year5/csc4006 Final Project/data/maintext_id_label.csv'

m_df = pd.read_csv(mtext_path,sep='\t')
m_df = m_df.dropna()
m_df = m_df[m_df.label != 'unknown']
m_df.shape

FileNotFoundError: ignored

In [0]:
m_df.head()

,paper_id,label,text
0,505487,neuroscience,Neural ensemble dynamics in dorsal motor cor...
1,508549,cancer biology,Title Proteasome inhibitor induced modulati...
2,508200,cell biology,Rescue of DNA damage in cells after constric...
3,508176,molecular biology,Title Fluorescence Microscopy of Piezo in ...
4,508325,biochemistry,Title Structural basis of sterol recognitio...


In [0]:
a_df.head()

,paper_id,label,text
0,505487,neuroscience,Speaking is a sensorimotor behavior whose neur...
1,505487,neuroscience,We recorded from electrode arrays in the 'hand...
2,505487,neuroscience,"Neurons in this area, which have not previousl..."
3,505487,neuroscience,This challenges whether the conventional model...
4,505487,neuroscience,Spoken words and syllables could be decoded fr...


In [0]:
##Dataframe cleanup methods 

##removes entries with length < length
def remove_sentences_by_length(length,dataframe):
  lSent = dataframe.loc[dataframe['text'].str.len() >= length]
  return lSent

##Remove null values from a dataframe
def remove_null(df):
  return df[pd.notnull(df.label)]

##cleans text in dataframe using clean_text function
def clean_dataframe(dataframe):
  cList = []
  for i,row in df.iterrows():
    text = row['text']
    label = row['label']
    clean_text = get_clean_text(text)
    cList.append((label,clean_text))
  return pd.DataFrame(cList,columns=['label','text'])

##Remove categories under a certain size
def remove_categories(min_length,dataframe):
  for lb in dataframe.label.unique():
    if(dataframe[dataframe.label == lb].shape[0] < min_length):
      dataframe = dataframe[dataframe.label != lb]
      print('remove',lb,'new df',dataframe.shape)
  return dataframe

def get_label_count(dataframe, label):
  return int(dataframe['label'].value_counts()[label])

##Sample a number of rows per category, if category len < sampleSize, get all of category
def sample_by_category(sampleSize,dataframe):
  sampled_df = pd.DataFrame(columns=['label','text','paper_index'])
  for lb in dataframe.label.unique():
    size = get_label_count(dataframe,lb)
    if size < sampleSize:
      sample_category_df = dataframe[dataframe.label == lb]
    else:
      sample_category_df = dataframe[dataframe.label == lb].sample(n=sampleSize)
      
    sampled_df = pd.concat([sampled_df,sample_category_df])
  return sampled_df

In [0]:
#Find average length
def get_average_length_of_text(dataframe):
  return dataframe.apply(lambda x: np.mean(x.str.len())).reset_index(name='mean_len_text')
  
  
##Get random piece of text from data set
def get_random_text(df):
  index = random.randint(1,df.shape[0])
  text = df.iloc[index]['text']
  print(text)
  return text

In [0]:
def remove_tab_etc(string):
  return ' '.join(string.split())

In [0]:
#m_df['paper_index'] = range(1, len(m_df) + 1)

In [0]:
#a_df['paper_index'] = range(1, len(a_df) + 1)

In [0]:
#a_df['paper_index'].dtype

dtype('int64')

In [0]:
##nltk tokenizer  funtions
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def split_into_sentences(dataframe):
  sent_df = pd.DataFrame(columns=['paper_id','text','label'])
  sList = []
  for i,row in dataframe.iterrows():
    text = row['text']
    label = row['label']
    paper_index = (row['paper_id'])
    sentences = sentence_tokenizer.tokenize(text)

    for s in sentences:
      s = remove_tab_etc(s)
      sList.append((paper_index,label,s))
  return pd.DataFrame(sList,columns=['paper_id','label','text'])

In [0]:
#m_df['paper_id'] = m_df['paper_id'].apply(lambda x: str(int(x)))
#a_df['paper_id'] = a_df['paper_id'].apply(lambda x: str(int(x)))

In [0]:
m_df = split_into_sentences(m_df)
#a_df = split_into_sentences(a_df)

In [0]:
m_df = remove_sentences_by_length(30,m_df)
#a_df = remove_sentences_by_length(20,a_df)

In [0]:
print(m_df.shape)
#print(a_df.shape)

(3550440, 3)


In [0]:
print(m_df.label.value_counts())
#print(a_df.label.value_counts())

genomics                                  100000
biochemistry                              100000
microbiology                              100000
neuroscience                              100000
cell biology                              100000
molecular biology                         100000
immunology                                100000
genetics                                  100000
evolutionary biology                      100000
biophysics                                100000
developmental biology                     100000
cancer biology                            100000
plant biology                             100000
bioinformatics                            100000
ecology                                   100000
systems biology                            89026
epidemiology                               69759
bioengineering                             56039
animal behavior and cognition              54140
physiology                                 36668
synthetic biology   

In [0]:
m_df = sample_by_category(100000,m_df)

In [0]:
a_df = sample_by_category(10000,a_df)

In [0]:
#a_df = a_df.sort_values(by=['paper_index'])
#m_df = m_df.sort_values(by=['paper_index'])

In [0]:
#a_df.to_csv('abstract_sentences_all', sep='\t', encoding='utf-8')
m_df.to_csv('maintext_sentences_all', sep='\t', encoding='utf-8')